# Read Excel Documents

Read MS Excel Documents with Python and the `openpyxl` library, and extract tables. The Excel was created with merged cells to make it a little bit more complex to process.

### Install the Required Libraries

In [ ]:
%pip install openpyxl pandas


### Python Imports

In [ ]:
import sys
sys.path.append('..\\code')

import os
from dotenv import load_dotenv
load_dotenv()

from docx import Document
import pandas as pd
from docx.shared import Inches
from docx.image.image import Image

from IPython.display import display, Markdown, HTML
from PIL import Image
from doc_utils import *
from utils.bcolors import bcolors as bc  


def show_img(img_path, width = None):
    if width is not None:
        display(HTML(f'<img src="{img_path}" width={width}>'))
    else:
        display(Image.open(img_path))

### Make sure we have the OpenAI Models information

We will need the GPT-4-Turbo and GPT-4-Vision models for this notebook.

When running the below cell, the values should reflect the OpenAI reource you have created in 

In [ ]:
model_info = {
        'AZURE_OPENAI_RESOURCE': os.environ.get('AZURE_OPENAI_RESOURCE'),
        'AZURE_OPENAI_KEY': os.environ.get('AZURE_OPENAI_KEY'),
        'AZURE_OPENAI_MODEL_VISION': os.environ.get('AZURE_OPENAI_MODEL_VISION'),
        'AZURE_OPENAI_MODEL': os.environ.get('AZURE_OPENAI_MODEL'),
}

model_info

### Code Definitions

Defining the function that will read in the docx file and return the text, images and tables as a list of strings, list of images and list of pandas dataframes respectively.

In [23]:

def get_excel_sheet_names(file_path):
    """
    Returns a list of sheet names from the specified Excel file.

    :param file_path: Path to the Excel file
    :return: List of sheet names
    """
    # Load the Excel file
    xls = pd.ExcelFile(file_path, engine='openpyxl')

    # Get the list of sheet names
    sheet_names = xls.sheet_names

    return sheet_names


def read_excel_to_dataframes(file_path):
    """
    Reads an Excel file and returns a dictionary of DataFrames.
    Each key in the dictionary corresponds to a sheet name,
    and each value is a DataFrame containing the data from that sheet.

    :param file_path: Path to the Excel file
    :return: Dictionary of DataFrames
    """
    # Load the Excel file
    xls = pd.ExcelFile(file_path, engine='openpyxl')

    # Dictionary to hold DataFrames
    dfs = {}

    # Read each sheet into a DataFrame
    for sheet_name in xls.sheet_names:
        dfs[sheet_name] = pd.read_excel(xls, sheet_name, header=None)
        # dfs[sheet_name] = dfs[sheet_name].ffill(axis=1)
        # dfs[sheet_name] = dfs[sheet_name].ffill(axis=0)

    return dfs


### Read Excel

Read the excel and print the dataframes out.

In [ ]:
# Usage
file_path = 'sample_data/hdgdemo/GladiusInvestorSample.xlsx'  
sheetname = get_excel_sheet_names(file_path)
print("sheets in file:", sheetname)
dataframes = read_excel_to_dataframes(file_path)

for sheet, df in dataframes.items():
    display(df)


### Print the Tables as Markdown

In [ ]:
for sheet, df in dataframes.items():
    display(Markdown(df.to_markdown()))


### Query the Table

Local RAG demo with the table extracted from particular excel sheet.

In [ ]:
from utils.openai_utils import ask_LLM


prompt = """
You are Investment Advisor trying to extract information form the table. Please review the following table of investment data. The table includes various fields such as investment type, amount, date, and performance metrics. As you review the data, consider the following questions:

Investment Type: Are all the investment types correctly categorized? Are there any discrepancies or inconsistencies?
Investment Amount: Are the amounts accurately recorded? Do they align with the expected values?
Date: Are the dates of the investments correctly noted? Are there any missing or incorrect dates?
Performance Metrics: Are the performance metrics (e.g., ROI, growth rate) accurately calculated and recorded? Are there any anomalies or outliers?
Overall Data Quality: Is the data complete and accurate? Are there any missing fields or errors that need to be addressed?

## START OF TABLE
{markdown}
## END OF TABLE

{query}

"""

def query_table(query,sheetname):
    p = prompt.format(markdown = dataframes[sheetname], query = query)
    output = ask_LLM(p, model_info=model_info)
    print(output)

querysheet = 'Gladius'
query_table('provide the investors details?',querysheet)

In [ ]:
from utils.openai_utils import ask_LLM


prompt = """
You are Investment Advisor trying to extract information form the table. Please review the following table of investment data. The table includes various fields such as investment type, amount, date, and performance metrics. As you review the data, consider the following questions:

Investment Type: Are all the investment types correctly categorized? Are there any discrepancies or inconsistencies?
Investment Amount: Are the amounts accurately recorded? Do they align with the expected values?
Date: Are the dates of the investments correctly noted? Are there any missing or incorrect dates?
Performance Metrics: Are the performance metrics (e.g., ROI, growth rate) accurately calculated and recorded? Are there any anomalies or outliers?
Overall Data Quality: Is the data complete and accurate? Are there any missing fields or errors that need to be addressed?

## START OF TABLE
{markdown}
## END OF TABLE

{query}

"""

def query_table(query):
    p = prompt.format(markdown = dataframes, query = query)
    output = ask_LLM(p, model_info=model_info)
    print(output)


query_table('provide all investors details for customer service')